# **CALREADER**

In [ ]:
!pip install nd2
!pip install numpy
!pip install pandas
!pip install openpyxl
!pip install pyinstaller auto-py-to-exe
!pip install nd2reader

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import nd2
from nd2reader import ND2Reader
import numpy as np
import pandas as pd
from PIL import Image, ImageTk, ImageDraw
import os
import datetime

In [21]:
class CALREADER:
    def __init__(self, root):
        self.root = root
        self.root.title("CALREADER")
        self.root.geometry("1600x1024")
        self.root.configure(bg='#474747')
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        
        self.ndfile = None
        self.nd2readerfile = None
        self.max_proj = None
        self.pixel_size_um = 0.1 #Temporary value
        self.roi_list = []
        self.df = None
        self.log_messages = []

        self.setup_ui()
        
        
    def setup_ui(self):
        main_frame = tk.Frame(self.root, bg='#474747')
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Canvas for max projection
        canvas_frame = tk.Frame(main_frame,bg='#474747')
        canvas_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0,10))
        
        self.canvas = tk.Canvas(canvas_frame, bg='black',highlightthickness=0)
        self.canvas_scroll = ttk.Scrollbar(canvas_frame, orient=tk.VERTICAL, command=self.canvas.yview)
        self.canvas_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        self.canvas.configure(yscrollcommand=self.canvas_scroll.set)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        self.canvas.bind('<Button-1>', self.on_click)
        self.canvas.bind('<Button-3>', self.delete_roi)

        self.create_log_window(main_frame)
        
        # 버튼 프레임
        button_frame = tk.Frame(self.root, bg='#474747')
        button_frame.pack(fill=tk.X, pady=10)
        
        # Import
        self.import_btn = tk.Button(button_frame, text="Import .nd2 file", 
                                   command=self.load_nd2, 
                                   font=('Arial', 12, 'bold'),
                                   bg='#121212', fg='white',  # bg=Background, fg=Font
                                   activebackground='#bababa',
                                   width=20, height=1,
                                   relief='flat', bd=0)
        self.import_btn.pack(side=tk.LEFT, padx=10)

        # Export Image
        self.export_img_btn = tk.Button(button_frame, text="Export ROI map image", 
                                       command=self.export_image_tif,
                                       font=('Arial', 12, 'bold'),
                                       bg='#121212', fg='white',
                                       width=20, height=1,
                                       relief='flat', bd=0)
        self.export_img_btn.pack(side=tk.RIGHT, padx=5)
        
        # Export xlsx
        self.export_btn = tk.Button(button_frame, text="Export datasheet", 
                                   command=self.export_excel,
                                   font=('Arial', 12, 'bold'),
                                   bg='#121212', fg='white',
                                   activebackground='#bababa',
                                   width=20, height=1,
                                   relief='flat', bd=0)
        self.export_btn.pack(side=tk.RIGHT, padx=10)

        self.root.after(50,self.instruction)

    def instruction(self):
        self.log_message("Welcome!\nSelect ROI(s) after importing .nd2 file.\nLeft click for ROI addition\nRight click for ROI deletion")
    
    def create_log_window(self,parent):
        log_frame = tk.Frame(parent, bg='#000000', relief='solid', bd=1, width=400)
        log_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=(10, 0))
        log_frame.pack_propagate(False)  # 고정 너비 유지
        
        # 로그 제목
        tk.Label(log_frame, text="Operation log", 
                 font=('Arial',12,'bold'), fg='white', bg='#000000').pack(anchor='w', padx=8, pady=(8,4))
        
        # 스크롤 가능한 텍스트
        text_frame = tk.Frame(log_frame, bg='#000000')
        text_frame.pack(fill=tk.BOTH, expand=True, padx=8, pady=(0,8))
        
        self.log_text = tk.Text(text_frame, bg='#121212', fg='#ffffff',
                               font=('Consolas', 10), wrap=tk.WORD,
                               insertbackground='white', state=tk.DISABLED,
                               relief='flat', bd=0)
        scrollbar = tk.Scrollbar(text_frame, orient=tk.VERTICAL, command=self.log_text.yview)
        self.log_text.configure(yscrollcommand=scrollbar.set)
        
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    def log_message(self, message, color='white'):
        self.log_messages.append(message)
        self.log_text.config(state=tk.NORMAL)
        
        # 타임스탬프 추가
        timestamp = datetime.datetime.now()
        self.log_text.insert(tk.END, '['+timestamp.strftime("%Y/%m/%d %H:%M:%S")+']' + '\n' + message + '\n', color)
        self.log_text.see(tk.END)  # 자동 스크롤
        self.log_text.config(state=tk.DISABLED)
        
        self.root.update_idletasks()  # 즉시 업데이트

    def clear_data(self):
        # 1. ND2 파일 닫기
        if hasattr(self, 'ndfile') and self.ndfile is not None:
            try:
                self.ndfile.close()
            except:
                pass
            self.ndfile = None
        
        # 2. 캔버스 초기화
        self.canvas.delete('all')
        self.log_message("Workspace cleared\n")
        
        # 3. 데이터 초기화
        self.max_proj = None
        self.photo = None
        self.roi_list.clear()
        self.df = None
        self.ndfile_path = None
        self.pixel_size_um = 0.1

    def load_nd2(self):
        self.clear_data()
        
        file_path = filedialog.askopenfilename(filetypes=[("ND2 files", "*.nd2")])
        if not file_path:
            return
        try:
            self.ndfile_path = file_path
            self.ndfile = nd2.ND2File(file_path)
            self.nd2readerfile = ND2Reader(file_path)
            # Pixel size 안전 추출
            try:
                px_size = self.ndfile.voxel_size().x
                self.pixel_size_um = px_size if px_size > 0 else 0.1
            except:
                self.pixel_size_um = 0.1
            
            # 전체 데이터 구조 확인
            full_data = self.ndfile.asarray()
            print(f"Full ND2 shape: {full_data.shape}")
            
            # 첫 채널 데이터 추출
            if len(full_data.shape) >= 3:
                ch0_data = full_data[:, 0, :, :] if len(full_data.shape) == 4 else full_data[0, :, :]
            else:
                ch0_data = full_data
            
            self.max_proj = np.max(ch0_data, axis=0).astype(np.uint8)
            
            # Display
            img = Image.fromarray(self.max_proj)
            img.thumbnail((900, 900), Image.Resampling.LANCZOS)
            self.photo = ImageTk.PhotoImage(img)
            
            self.canvas.delete('all')
            self.canvas.create_image(0, 0, image=self.photo, anchor='nw')
            self.canvas.configure(scrollregion=self.canvas.bbox('all'))
            
            self.roi_list.clear()

        except Exception as e:
            messagebox.showerror("Error", str(e))
            import traceback
            print(traceback.format_exc())

        self.log_message("File imported" + f"Shape: {self.max_proj.shape}\n"
                                       +f"Pixel size: {self.pixel_size_um:.3f}μm\n"
                                      +f"T frames: {ch0_data.shape[0] if len(ch0_data.shape)>2 else 1}")
    
    def on_click(self, event):
        if self.max_proj is None:
            return    
    # Canvas coordinates to image coordinates
        photo_w = self.photo.width()   # 메서드 호출
        photo_h = self.photo.height()  # 메서드 호출
        x = int(event.x * self.max_proj.shape[1] / photo_w)
        y = int(event.y * self.max_proj.shape[0] / photo_h)
        x = max(0, min(x, self.max_proj.shape[1]-1))
        y = max(0, min(y, self.max_proj.shape[0]-1))
    
    # 0.5um radius ROI in pixels
        roi_radius_px = max(1, int(0.25 / self.pixel_size_um))  # 최소 1px 보장
    
        self.roi_list.append((x, y))
        roi_idx = len(self.roi_list)
        
    # Draw circle on canvas
        scale_x = photo_w / self.max_proj.shape[1]
        scale_y = photo_h / self.max_proj.shape[0]
        cx = x * scale_x
        cy = y * scale_y
        r = roi_radius_px * min(scale_x, scale_y)
        self.canvas.create_oval(cx-r, cy-r, cx+r, cy+r, outline='red', width=3, tags=f'roi_{roi_idx}')
     # Mark ROI number on canvas   
        text_y = cy - r - 5
        roi_text_id = self.canvas.create_text(cx, text_y, 
                                        text=str(roi_idx), 
                                        fill='red',
                                        font=('Arial', 10),
                                        tags=f'roi_text_{roi_idx}')
    
        print(f"ROI {len(self.roi_list)} added: ({x}, {y}) px → {x*self.pixel_size_um:.2f}, {y*self.pixel_size_um:.2f} μm")
        self.log_message(f"ROI {len(self.roi_list)} added")
        

    def delete_roi(self, event): #우클릭
        photo_w = self.photo.width()
        photo_h = self.photo.height()
        click_x = int(event.x * self.max_proj.shape[1] / photo_w)
        click_y = int(event.y * self.max_proj.shape[0] / photo_h)
        
        closest_idx = None
        min_dist = float('inf')
        
        # 30px 이내 가장 가까운 ROI 찾기
        for i, (rx, ry) in enumerate(self.roi_list):
            dist = ((click_x - rx)**2 + (click_y - ry)**2)**0.5
            if dist < min_dist and dist < 30:  
                min_dist = dist
                closest_idx = i
        
        if closest_idx is not None:
            # ROI와 텍스트 삭제
            self.canvas.delete(f'roi_{closest_idx+1}')
            self.canvas.delete(f'roi_text_{closest_idx+1}')
            #del self.roi_list[closest_idx]
            print(f"ROI {closest_idx+1} 삭제됨")
        
    def analyze_rois(self):
        if self.ndfile is None:
            return
        
        ch0_data = self.ndfile.asarray()[:, 0, :, :]  # T, Y, X
        
        # 차원 확인 및 처리
        if len(ch0_data.shape) == 4:  # T, C, Y, X
            data_3d = ch0_data[:, 0, :, :]  # T, Y, X
        elif len(ch0_data.shape) == 3:  # T, Y, X
            data_3d = ch0_data
        else:  # 2D 이미지인 경우
            messagebox.showwarning("경고", "Time series missing")
            return
            
        print(f"Processing 3D data shape: {data_3d.shape}")  # T, Y, X
        
        roi_data = []
        for i, (cx, cy) in enumerate(self.roi_list):
            roi_radius_px = max(1, int(0.25 / self.pixel_size_um))  # 일관성
            
            y_slice = slice(max(0, cy-roi_radius_px), min(data_3d.shape[2], cy+roi_radius_px))
            x_slice = slice(max(0, cx-roi_radius_px), min(data_3d.shape[1], cx+roi_radius_px))
            
            roi_max_t = np.max(data_3d[:, y_slice, x_slice], axis=(1,2))
            roi_data.append(roi_max_t)

        time_col = 'Time(s)'
            
        timestamps = self.nd2readerfile.get_timesteps()/1000
        
        self.df = pd.DataFrame({
            time_col: timestamps,
            **{f'ROI_{i+1}_MaxInt': data for i, data in enumerate(roi_data)}
        })
        

    def export_image_tif(self):
        if self.max_proj is None or not self.roi_list:
            messagebox.showwarning("경고", "File not loaded")
            return
        
        file_path = filedialog.asksaveasfilename(
            defaultextension=".tif",
            filetypes=[("TIFF files", "*.tif"), ("PNG files", "*.png")],
            title="Save ROI map"
        )
        if not file_path:
            return
        
        try:
            with nd2.ND2File(self.ndfile_path) as ndfile:  # 경로 재사용
                full_data = ndfile.asarray()
                if len(full_data.shape) >= 3:
                    ch0_data = full_data[:, 0, :, :] if len(full_data.shape) == 4 else full_data[0, :, :]
                else:
                    ch0_data = full_data
                
                # Max projection 재계산 (16bit 원본)
                original_max_proj = np.max(ch0_data, axis=0)
            
            print(f"max proj range: {original_max_proj.min()} - {original_max_proj.max()}")
            
            # RGB 이미지로 변환 (ROI 그리기용)
            # 정규화해서 시각화 (0-65535 → 0-255)
            norm_img = np.clip((original_max_proj / np.max(original_max_proj)) * 255, 0, 255).astype(np.uint8)
            rgb_img = Image.fromarray(norm_img).convert('RGB')
            
            # ROI 원 그리기
            draw = ImageDraw.Draw(rgb_img)
            pixel_size_px = 1 / self.pixel_size_um
            
            for i, (x, y) in enumerate(self.roi_list):
                roi_radius_px = max(3, int(0.25 * pixel_size_px))
                bbox = [x-roi_radius_px, y-roi_radius_px, x+roi_radius_px, y+roi_radius_px]
                draw.ellipse(bbox, outline='red', width=4)
                draw.text((x+5, y-10), f'{i+1}', fill='red')  # ROI 번호 추가
            
            # 저장 (RGB로 저장 → 모든 이미지 뷰어 호환)
            rgb_img.save(file_path)
            self.log_message(f"Image of {len(self.roi_list)} ROI(s) saved at\n{file_path}")
            
        except Exception as e:
            messagebox.showerror("오류", f"저장 실패: {str(e)}")

    def calculation(self, window_size=24, percentile=30):
        if self.df is None or len(self.roi_list) == 0:
            self.log_message("Data not loaded")
            return False
        
        try:
            window_size = int(window_size)
            percentile = float(percentile)
            
            # ROI 컬럼만 추출 (Frame 제외)
            roi_columns = [col for col in self.df.columns if col.startswith('ROI_')]
            n_rois = len(roi_columns)
            
            # baseline 배열 초기화 (T x N_ROI)
            self.baseline_values = np.zeros((len(self.df), n_rois))
            
            # 각 ROI별 baseline 계산
            for roi_idx, col in enumerate(roi_columns):
                roi_data = self.df[col].values
                
                for i in range(len(roi_data)):
                    # 슬라이딩 윈도우
                    if i < window_size:
                        window = roi_data[i:i+window_size]
                    else:
                        window = roi_data[max(0, i-window_size):i+1]
                    
                    # percentile로 baseline 설정
                    self.baseline_values[i, roi_idx] = np.percentile(window, percentile)
            
            # NaN/Inf 처리 (평균값으로 대체)
            mean_baseline = np.nanmean(self.baseline_values, axis=0)
            self.baseline_values = np.nan_to_num(self.baseline_values, 
                                               nan=mean_baseline, 
                                               posinf=mean_baseline, 
                                               neginf=mean_baseline)
            
            # ΔF/F 계산
            F0 = np.mean(self.baseline_values, axis=0)  # 각 ROI의 평균 baseline
            self.df_deltaf = self.df[roi_columns].div(F0, axis=1) - 1  # (F-F0)/F0
            
            return True
            
        except Exception as e:
            self.log_message(f"Baseline calculation failed: {str(e)}")
            return False
    
    def export_excel(self):
    # Analyze ROIs
        if len(self.roi_list) >= 1:
            self.analyze_rois()
        if self.df is None or len(self.roi_list) == 0:
            return
        if not self.calculation(window_size=50, percentile=30):
            return

        timestamps = self.nd2readerfile.get_timesteps()/1000
        
        file_path = filedialog.asksaveasfilename(
            defaultextension=".xlsx",
            filetypes=[("Excel files", "*.xlsx")],
            title="Export 분석 결과"
        )
        if not file_path:
            return
        
        try:
            with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
                # 시트 1: Raw Data
                self.df.to_excel(writer, sheet_name='Raw_Data', index=False)
                
                # 시트 2: Baseline (F0)
                baseline_df = pd.DataFrame(self.baseline_values, 
                                         columns=[f'ROI_{i+1}_F0' for i in range(len(self.roi_list))])
                baseline_df['Time(s)'] = timestamps
                baseline_df = baseline_df[['Time(s)'] + [col for col in baseline_df.columns if col != 'Time(s)']]
                baseline_df.to_excel(writer, sheet_name='Baseline', index=False)
                
                # 시트 3: ΔF/F
                deltaf_df = self.df[['Time(s)']].join(
                    pd.DataFrame(self.df_deltaf.values, 
                               columns=[f'ROI_{i+1}_dFoverF' for i in range(len(self.roi_list))]))
                deltaf_df.to_excel(writer, sheet_name='DeltaF_F', index=False)
            
            self.log_message(f"Datasheet saved at \n{file_path}")
            
        except Exception as e:
            self.log_message(f"Excel 저장 실패: {str(e)}")
            messagebox.showerror("오류", str(e))
        
    def on_closing(self): 
    #Clearing when shutting down
        if hasattr(self, 'ndfile') and self.ndfile is not None:
            try:
                self.ndfile.close()
            except:
                pass
        self.root.destroy()

In [22]:
root = tk.Tk()
app = CALREADER(root)
root.mainloop()